In [1]:
from bs4 import BeautifulSoup
import requests
import unicodedata
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [32]:
# Helper Functions

def clean(s, z):
    s = s.replace('\n' ,'')
    s = s.replace('\t' ,'')
    if(z==1): # Cleaning Country & Political_Party
        s = s.split(":")
        return s[-1]
    elif(z==2):
        s = s.split("<")
        return s[0]
    else :
        return s
    
def decodeEmail(e):
    de = ""
    k = int(e[:2], 16)

    for i in range(2, len(e)-1, 2):
        de += chr(int(e[i:i+2], 16)^k)

    return de

def get_html_body_code(url):
    url = url+('/html_body')
    r1 = requests.get(url)
    s = BeautifulSoup(r1.content, 'lxml')
    body_tag = s.find('body')

    return body_tag

def get_plain_text(url):
    
    r2 = requests.get(url+'#plain-text')
    s = BeautifulSoup(r2.content)
    
    plain_text  = s.find(class_='message-text')
    if(plain_text == None):
        return None
    plain_text = plain_text.text
    
    plain_text = unicodedata.normalize('NFKD', plain_text)
    encoded_text = plain_text.encode('utf-8')    
    decoded_text = encoded_text.decode('utf-8')

    cleaned_text = unicodedata.normalize('NFKD', decoded_text)

    cleaned_text = ''.join(c for c in cleaned_text if unicodedata.category(c)[0] != 'C')


    return cleaned_text



In [38]:
def scrap_message(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    
    
    info_box =soup.find( class_='connections-box')
    elements = info_box.find('ul').find_all('li')

    org_name = clean(elements[0].get_text(),1)
    Political_Party = clean(elements[1].get_text(),1)
    Country = clean(elements[2].get_text(),1)
    title = clean(soup.find(class_='page-title').get_text() ,0)
    
    upper_info = soup.find_all(class_='content-box-meta__value')
    
    from_text = clean(upper_info[0].text,2)
    
    org_site = upper_info[0].a
    if(org_site.span == None):
        org_site = org_site['data-cfemail']
    else:
         org_site = org_site.span['data-cfemail']
            
    org_site = decodeEmail(org_site)
    subject =upper_info[1].text.strip()
    send_date = clean(upper_info[2].text,2)
    html_body_code = get_html_body_code(url)
    plain_text = get_plain_text(url)
    
    
    message = {}
    message['Country'] = Country
    message['Political Party']=Political_Party
    message['Organization Name'] = org_name
    message['Organization website'] = org_site
    message['From'] = from_text
    message['Subject Line']=subject
    message['Send Date'] = send_date
    message['HTML'] = html_body_code
    message['Plain Text'] = plain_text
    
    return message

In [39]:
home_url = 'https://politicalemails.org/messages'
response = requests.get(home_url)
soup = BeautifulSoup(response.content, 'lxml')

In [40]:
all_links = soup.find(class_='resource-list').find_all('a')
all_links = [link['href'] for link in all_links]

In [41]:
all_links

['https://politicalemails.org/messages/1015162',
 'https://politicalemails.org/messages/1015161',
 'https://politicalemails.org/messages/1015160',
 'https://politicalemails.org/messages/1015159',
 'https://politicalemails.org/messages/1015158',
 'https://politicalemails.org/messages/1015156',
 'https://politicalemails.org/messages/1015157',
 'https://politicalemails.org/messages/1015154',
 'https://politicalemails.org/messages/1015155',
 'https://politicalemails.org/messages/1015153',
 'https://politicalemails.org/messages/1015152',
 'https://politicalemails.org/messages/1015151',
 'https://politicalemails.org/messages/1015150',
 'https://politicalemails.org/messages/1015149',
 'https://politicalemails.org/messages/1015148',
 'https://politicalemails.org/messages/1015146',
 'https://politicalemails.org/messages/1015145',
 'https://politicalemails.org/messages/1015144',
 'https://politicalemails.org/messages/1015143',
 'https://politicalemails.org/messages/1015147',
 'https://politicale

In [46]:
main_page_mails = []
for email in all_links:
    try:
        scrapped_email = scrap_message(email)
        main_page_mails.append(scrapped_email)
    except:
        print(email)

# All mails are scrapped succefully without errors
main_page_mails = pd.DataFrame(main_page_mails)

In [51]:
main_page_mails

,Country,Political Party,Organization Name,Organization website,From,Subject Line,Send Date,HTML,Plain Text
0,n/a,n/a,n/a,dnr.updates@updates.mndnr.gov,MN Department of Natural Resources,Red Flag Warning Begins at 11:00 a.m.,"April 13, 2023 11:00 AM","[\n, [if (gte mso 9)|(IE)]>\r\n <table style=...",header *DNR News Release**FOR IMMEDIATE RELEA...
1,United States,n/a,Children's National Health System,riseandshine@childrensnational.org,Rise and Shine,Celebrating Autism Acceptance,"April 13, 2023 11:00 AM","[\n, \n, \n, [if !gte mso 9]><!--, [Plus: Ever...",Plus: Everything you need to know about autism...
2,n/a,n/a,n/a,hello@mail.fightwithkash.com,Fight with Kash,America’s Degrading National Security Under Biden,"April 13, 2023 11:00 AM",[[Kash talks about the recent classified docum...,Kash talks about the recent classified documen...
3,n/a,n/a,n/a,nathan@unwaveringfreedom.com,"Lt. Gen. Jack Bergman, USMC (ret)",We didn’t Start the Fire..But we tried to Figh...,"April 13, 2023 11:00 AM","[\n, [\n, [It’s like a Billy Joel song.], \n, ...",It’s like a Billy Joel song. ͏ ͏ ͏ ͏ ͏ ͏ ͏ ͏...
4,n/a,n/a,n/a,noreply@medium.com,Medium Daily Digest,Boost Your Speaking Fluency with “4–3–2” | Fab...,"April 13, 2023 11:00 AM","[[[<tbody><tr><td class=""l n"" style=""width: 10...",Stories for Jack xxxxxx@xxxxxx ([link removed]...
5,n/a,n/a,n/a,contact@mooney-emails.com,Alex X. Mooney,The Swamp’s latest attack,"April 13, 2023 10:59 AM","[\n, [\r\r\nWe can’t fall behind with our race...","<[link removed]>John, After we paused fundrai..."
6,United States,n/a,Texas Scorecard,Michael@TexasScorecard.com,Michael Quinn Sullivan,Texas Minute: 4/13/2023,"April 13, 2023 10:59 AM","[\n, <![endif], [Smuggling aliens, challenging...",ACCOUNT [[link removed]]READ ONLINE [link remo...
7,n/a,n/a,n/a,Franson@TeamFranson.com,Mary Franson,Democrats and assisted suicide,"April 13, 2023 10:44 AM","[\n, \n, \n, [if !gte mso 9]><!--, [It's dark ...",It's dark in MinnesotaView this email in your ...
8,United States,n/a,Pop Acta,news@americanupdate.com,Weiss Ratings,Banking Nightmare,"April 13, 2023 10:39 AM","[[], [], \n, Pre-header Preview Text Begin , ...",During and after the Great Financial Crisis of...
9,United States,n/a,Pop Acta,news@e.americanactionnews.com,Margaret Mary Dunn,My police officer son is facing life in prison,"April 13, 2023 10:39 AM","[\n, Pre-header Preview Text Begin , \n, [I'm...",None


In [54]:
main_page_mails.to_csv('Main_page_scrapped.csv')